In [59]:
import pandas as pd
from prophet import Prophet

In [60]:
df = pd.read_csv('/content/simple - Sheet1.csv')
df.head()

,BillingDate,NetValue
0,01/01/2022,1000
1,01/02/2022,2000
2,01/03/2022,3000
3,01/04/2022,4000
4,01/05/2022,5000


In [61]:
print(df.dtypes)

BillingDate    object
NetValue        int64
dtype: object


In [27]:
df['Net Value'] = df['Net Value'].map(lambda x: float(x))

In [ ]:
#df1=df.loc[df['Organization ID'] == 1]
#df11=df1.loc[df['Material Number'] == 500001]
#df12=df11.loc[df['Customer Number'] == 100001]
#df2=df12.loc[df['Region'] == 'TX']

#df2.head()

,Organization ID,Billing Date,Customer Number,Material Number,Region,Net Value
0,1,01/01/2021,100001,500001,TX,"19,569.06"


In [54]:
df2 = df[['Net Value','Billing Date']]
df2.head()


,Net Value,Billing Date
0,19569.06,01/01/2021
1,641.36,01/01/2021
2,16.86,01/01/2021
3,335.63,01/01/2021
4,2262.15,01/01/2021


In [56]:
d = df2.groupby('Billing Date').sum()

In [57]:
d.head()


,Net Value
Billing Date,
01/01/2021,940008.55
01/03/2021,1424101.03
01/04/2021,661954.96
01/05/2021,1446287.61
01/06/2021,347987.94


In [58]:
d.to_frame()

AttributeError: ignored

In [47]:
df4 = df3.rename(columns={'Billing Date':'ds','Net Value':'y'})
df4.head()

,y
Billing Date,
01/01/2021,940008.55
01/03/2021,1424101.03
01/04/2021,661954.96
01/05/2021,1446287.61
01/06/2021,347987.94


In [48]:
df4.head()

,y
Billing Date,
01/01/2021,940008.55
01/03/2021,1424101.03
01/04/2021,661954.96
01/05/2021,1446287.61
01/06/2021,347987.94


In [49]:
print(df4.dtypes)

y    float64
dtype: object


In [62]:
df['Year'] = df['BillingDate'].apply(lambda x: str(x)[-4:])
df['Day'] = df['BillingDate'].apply(lambda x: str(x)[3:5])
df['Month'] = df['BillingDate'].apply(lambda x: str(x)[:2])
df['ds'] = pd.DatetimeIndex(df['Year']+'-'+df['Month']+'-'+df['Day'])


In [63]:
df.drop(['BillingDate', 'Year', 'Month', 'Day'], axis=1, inplace=True)

In [64]:
df.head()

,NetValue,ds
0,1000,2022-01-01
1,2000,2022-01-02
2,3000,2022-01-03
3,4000,2022-01-04
4,5000,2022-01-05


In [65]:
df5 = df.rename(columns={'NetValue':'y'})

In [66]:
df5.head()

,y,ds
0,1000,2022-01-01
1,2000,2022-01-02
2,3000,2022-01-03
3,4000,2022-01-04
4,5000,2022-01-05


In [67]:
print(df5.dtypes)

y              int64
ds    datetime64[ns]
dtype: object


In [ ]:
df5['y'] = pd.to_numeric(df5['y'],errors = 'coerce')

In [68]:
prophet = Prophet()
prophet.fit(df5)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 8.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpxdqn5py3/zl6s_5wg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpxdqn5py3/ok8wockn.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=49128', 'data', 'file=/tmp/tmpxdqn5py3/zl6s_5wg.json', 'init=/tmp/tmpxdqn5py3/ok8wockn.json', 'output', 'file=/tmp/tmpoj0_82xx/prophet_model-20220830120916.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:09:16 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] 

In [69]:
future = prophet.make_future_dataframe(periods=100)
future.tail()

,ds
107,2022-04-18
108,2022-04-19
109,2022-04-20
110,2022-04-21
111,2022-04-22


In [70]:
forecast = prophet.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
107,2022-04-18,108000.038509,108000.029422,108000.047781
108,2022-04-19,109000.038873,109000.029624,109000.048430
109,2022-04-20,110000.039236,110000.029802,110000.048859
110,2022-04-21,111000.039600,111000.030040,111000.049380
111,2022-04-22,112000.039964,112000.030272,112000.049978


In [71]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

,ds,yhat,yhat_lower,yhat_upper
0,2022-01-01,999.999600,999.999600,999.999600
1,2022-01-02,1999.999964,1999.999964,1999.999964
2,2022-01-03,3000.000327,3000.000327,3000.000327
3,2022-01-04,4000.000691,4000.000691,4000.000691
4,2022-01-05,5000.001055,5000.001055,5000.001055


In [73]:
forecast.to_csv("simple_res_final.csv")